In [6]:
pip install requests beautifulsoup4

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.0 MB/s eta 0:00:00
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/49/48/b89a9ccc78ea7a2a0b37c20a912b98c840210f277747e2380ee8d72784cc/charset_normalizer-3.3.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached charset_normalizer-3.3.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/d2/b2/b157855192a68541a91ba7b2bbcb91f1b4faa51f8bae38d8005c034be524/urllib3-2.0.7-py3-none-any.whl.metadata
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6

In [20]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://www.kiesraad.nl/adviezen-en-publicaties/formulieren/2016/5/kieskringen/kieskringen-en-hoofdstembureaus-tweede-kamerverkiezingen-en-europees-parlementsverkiezingen'
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
kieskring_data = soup.find('tbody').get_text().splitlines()

kieskringen = kieskring_data[4::5]
gemeenten_raw = kieskring_data[3::5]

gemeenten = []
dict = {}

with open('../kadaster/cbs_gemeentes.json') as fs:
    cbs = json.load(fs)

gemeenten = {}

print(kieskringen)
print(gemeenten_raw)

['Groningen', 'Leeuwarden', 'Assen', 'Zwolle', 'Lelystad', 'Nijmegen', 'Arnhem', 'Utrecht', 'Amsterdam', 'Haarlem', 'Den Helder', "'s-Gravenhage", 'Rotterdam', 'Dordrecht', 'Leiden', 'Middelburg', 'Tilburg', '’s-Hertogenbosch', 'Maastricht', 'Bonaire']
['De provincie Groningen.', 'De provincie Fryslân.', 'De provincie Drenthe.', 'De provincie Overijssel.', 'De provincie Flevoland.', 'De gemeenten Berg en Dal, Beuningen, Buren, Culemborg, Druten, Heumen, Maasdriel, Neder-Betuwe, Nijmegen, Tiel, West Betuwe, West Maas en Waal, Wijchen, Zaltbommel.', 'De gemeenten van de provincie Gelderland die niet toe kieskring 6 behoren.', 'De provincie Utrecht.', 'De gemeente Amsterdam.', 'De gemeenten Aalsmeer, Amstelveen, Beverwijk, Blaricum, Bloemendaal, Diemen, Gooise Meren, Haarlem, Haarlemmermeer, Heemskerk, Heemstede, Hilversum, Huizen, Laren, Ouder-Amstel, Uithoorn, Velsen,\xa0Wijdemeren, Zandvoort.', 'De gemeenten van de provincie Noord-Holland die niet tot de kieskringen 9 of 10 behoren.', 

In [44]:
import re

# Alle gemeenten per kieskring in een aparte lijst zetten en wat nutteloze
# tekst aan het begin van iedere lijst verwijderen
for i, el in enumerate(gemeenten_raw[:-1]):
    if 'De provincie' in el:
        for gem in cbs:
            if gem['Naam_27'] == el.replace('De provincie ', '').replace('.', ''):
                gemeenten[gem['Naam_2']] = kieskringen[i]
    elif 'De gemeenten van de' in el:
        provincie = re.search(r"provincie (?P<provincie>\S+)", el).group(1)
        for gem in cbs:
            if gem['Naam_27'] == provincie and gem['Naam_2'] not in gemeenten:
                gemeenten[gem['Naam_2']] = kieskringen[i]
    elif 'De gemeenten ' in el:
        gems = el.replace('De gemeenten ', '').replace('.', '')
        for gem in gems.split(', '):
            gemeenten[gem] = kieskringen[i]
    elif 'De gemeente Amsterdam' in el:
        gemeenten['Amsterdam'] = kieskringen[i]
    elif 'De gemeente Rotterdam' in el:
        gemeenten['Rotterdam'] = kieskringen[i]
    elif "De gemeente 's-G" in el:
        gemeenten["'s-Gravenhage"] = kieskringen[i]
        gemeenten["Buitenland"] = kieskringen[i]
    else:
        print(el)
print(gemeenten)

{'Eemsdelta': 'Groningen', 'Groningen': 'Groningen', 'Het Hogeland': 'Groningen', 'Midden-Groningen': 'Groningen', 'Oldambt': 'Groningen', 'Pekela': 'Groningen', 'Stadskanaal': 'Groningen', 'Veendam': 'Groningen', 'Westerkwartier': 'Groningen', 'Westerwolde': 'Groningen', 'Achtkarspelen': 'Leeuwarden', 'Ameland': 'Leeuwarden', 'Dantumadiel': 'Leeuwarden', 'De Fryske Marren': 'Leeuwarden', 'Harlingen': 'Leeuwarden', 'Heerenveen': 'Leeuwarden', 'Leeuwarden': 'Leeuwarden', 'Noardeast-Fryslân': 'Leeuwarden', 'Ooststellingwerf': 'Leeuwarden', 'Opsterland': 'Leeuwarden', 'Schiermonnikoog': 'Leeuwarden', 'Smallingerland': 'Leeuwarden', 'Súdwest-Fryslân': 'Leeuwarden', 'Terschelling': 'Leeuwarden', 'Tytsjerksteradiel': 'Leeuwarden', 'Vlieland': 'Leeuwarden', 'Waadhoeke': 'Leeuwarden', 'Weststellingwerf': 'Leeuwarden', 'Aa en Hunze': 'Assen', 'Assen': 'Assen', 'Borger-Odoorn': 'Assen', 'Coevorden': 'Assen', 'Emmen': 'Assen', 'Hoogeveen': 'Assen', 'Meppel': 'Assen', 'Midden-Drenthe': 'Assen', 'N

In [46]:
for gem in cbs:
    if gem['Naam_2'] not in gemeenten:
        print(gem['Naam_2'])

In [54]:
for gem in ['Bonaire', 'Sint Eustatius', 'Saba']:
    gemeenten[gem] = 'Bonaire'

In [56]:
print(gemeenten)

with open('gemeenten-kieskringen.json', 'w', encoding='utf8') as fp:
	json.dump(gemeenten, fp, ensure_ascii=False, indent=2)

{'Eemsdelta': 'Groningen', 'Groningen': 'Groningen', 'Het Hogeland': 'Groningen', 'Midden-Groningen': 'Groningen', 'Oldambt': 'Groningen', 'Pekela': 'Groningen', 'Stadskanaal': 'Groningen', 'Veendam': 'Groningen', 'Westerkwartier': 'Groningen', 'Westerwolde': 'Groningen', 'Achtkarspelen': 'Leeuwarden', 'Ameland': 'Leeuwarden', 'Dantumadiel': 'Leeuwarden', 'De Fryske Marren': 'Leeuwarden', 'Harlingen': 'Leeuwarden', 'Heerenveen': 'Leeuwarden', 'Leeuwarden': 'Leeuwarden', 'Noardeast-Fryslân': 'Leeuwarden', 'Ooststellingwerf': 'Leeuwarden', 'Opsterland': 'Leeuwarden', 'Schiermonnikoog': 'Leeuwarden', 'Smallingerland': 'Leeuwarden', 'Súdwest-Fryslân': 'Leeuwarden', 'Terschelling': 'Leeuwarden', 'Tytsjerksteradiel': 'Leeuwarden', 'Vlieland': 'Leeuwarden', 'Waadhoeke': 'Leeuwarden', 'Weststellingwerf': 'Leeuwarden', 'Aa en Hunze': 'Assen', 'Assen': 'Assen', 'Borger-Odoorn': 'Assen', 'Coevorden': 'Assen', 'Emmen': 'Assen', 'Hoogeveen': 'Assen', 'Meppel': 'Assen', 'Midden-Drenthe': 'Assen', 'N